# Subjektive Leistungseinschätzung


## Import

In [34]:
import plotly.io as pio
import pandas as pd
import plotly.express as px

## Template

In [35]:
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

## Daten einlesen

In [36]:
file_path = "data.xlsx"
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Remove first (irrelevant) sheet
sheets.pop(next(iter(sheets)))

leistung_df = {}

# Loop through all years
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()

    # Identify Bachelor and Master columns
    studiengaenge = [col for col in df.columns if "Bachelor" in col or "Master" in col]

    # Select relevant rows
    positiv_row = df[
        (df["Variable"] == "Subjektive Leistungseinschätzung")
        & (df["Category"] == "Positiv Einschätzung (%)")
    ]
    indifferent_row = df[
        (df["Variable"] == "Subjektive Leistungseinschätzung")
        & (df["Category"] == "Indifferent (%)")
    ]
    negativ_row = df[
        (df["Variable"] == "Subjektive Leistungseinschätzung")
        & (df["Category"] == "Negativ Einschätzung (%)")
    ]
    total_students_row = df[
        (df["Variable"] == "Subjektive Leistungseinschätzung")
        & (df["Category"] == "Anzahl")
    ]

    # Use only shared columns starting from the 3rd column
    common_columns = positiv_row.columns.intersection(total_students_row.columns)[2:]

    # Extract and clean values
    positiv_values = positiv_row[common_columns].astype(float).fillna(0).squeeze()
    indifferent_values = indifferent_row[common_columns].astype(float).fillna(0).squeeze()
    negativ_values = negativ_row[common_columns].astype(float).fillna(0).squeeze()
    total_students_values = total_students_row[common_columns].astype(float).fillna(0).squeeze()

    # Calculate absolute values
    abs_positiv = (positiv_values / 100) * total_students_values
    abs_indifferent = (indifferent_values / 100) * total_students_values
    abs_negativ = (negativ_values / 100) * total_students_values

    # Sum up across all study programs
    total_abs_positiv = abs_positiv.sum()
    total_abs_indifferent = abs_indifferent.sum()
    total_abs_negativ = abs_negativ.sum()
    total_students = total_students_values.sum()

    # Calculate weighted percentages
    gewichteter_positiv = (total_abs_positiv / total_students) * 100 if total_students > 0 else 0
    gewichteter_indifferent = (total_abs_indifferent / total_students) * 100 if total_students > 0 else 0
    gewichteter_negativ = (total_abs_negativ / total_students) * 100 if total_students > 0 else 0

    # Store results in dictionary
    leistung_df[jahr] = {
        "Gewichteter Positiv (%)": gewichteter_positiv,
        "Gewichteter Indifferent (%)": gewichteter_indifferent,
        "Gewichteter Negativ (%)": gewichteter_negativ
    }

# Convert dictionary to DataFrame
df_leistung = pd.DataFrame.from_dict(leistung_df, orient="index").reset_index()
df_leistung.rename(columns={"index": "Jahr"}, inplace=True)
df_leistung = df_leistung.sort_values(by="Jahr")

In [37]:
# Prepare data for stacked bar plot
df_leistung_melted = df_leistung.melt(
    id_vars="Jahr",
    var_name="Einschätzung",
    value_name="Prozent"
)

# Define color mapping
color_map = {
    "Positiv": "#2ca02c",        # Grün
    "Indifferent": "#ffcc00",    # Gelb
    "Negativ": "#d62728"         # Rot
}

# Mapping for shorter category names
label_map = {
    "Gewichteter Positiv (%)": "Positiv",
    "Gewichteter Indifferent (%)": "Indifferent",
    "Gewichteter Negativ (%)": "Negativ"
}

# Apply label mapping
df_leistung_melted["Einschätzung"] = df_leistung_melted["Einschätzung"].map(label_map)

# Create stacked bar chart
fig = px.bar(
    df_leistung_melted,
    x="Jahr",
    y="Prozent",
    color="Einschätzung",
    barmode="stack",
    title="Seit 2021 deutlich weniger negative Selbsteinschätzungen",
    subtitle="Entwicklung der Studierendenbefragung im FB09 zwischen 2013 und 2024",
    color_discrete_map=color_map,
    template="infoviz"
)

# Layout configuration
fig.update_layout(
    yaxis=dict(range=[0, 100], title="Anteil Selbsteinschätzung (%)"),
    xaxis=dict(title="", dtick=1),
    height=500,
    width=1500,
    legend=dict(
        title_text="",
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="left",
        x=0,
        font=dict(family="Arial", weight="bold", color="grey")
    )
)

fig.show()
fig.write_image("Plots/pdf/leistung1.pdf")

In [41]:
df_leistung_filtered = df_leistung[df_leistung["Jahr"].isin(["2019", "2020", "2021", "2022", "2023", "2024"])]

df_leistung_melted = df_leistung_filtered.melt(id_vars="Jahr", var_name="Einschätzung", value_name="Prozent")
df_leistung_melted = df_leistung_melted[df_leistung_melted["Einschätzung"].isin(
    ["Gewichteter Positiv (%)", "Gewichteter Indifferent (%)", "Gewichteter Negativ (%)"]
)]

color_map = {
    "Gewichteter Positiv (%)": "#A9A9A9",   # Grau
    "Gewichteter Indifferent (%)": "#ffcc00",  # Gelb
    "Gewichteter Negativ (%)": "#d62728"   # Rot
}

fig = px.line(
    df_leistung_melted,
    x="Jahr",
    y="Prozent",
    color="Einschätzung",
    markers=False,
    title="Trendumkehr 2021/2022 – Weniger Studierende schätzen ihre Leistung negativ ein",
    subtitle="Entwicklung der Studierendenbefragung im FB09 zwischen 2019 und 2024",
    color_discrete_map=color_map,
    template="infoviz"
)

for jahr in ["2021", "2022"]:
    fig.add_shape(
        type="line",
        x0=jahr, x1=jahr,
        y0=0, y1=100,
        line=dict(color="grey", width=2, dash="dash"),                                                                                                                                                                                                                                                                                                                                                                                              
    )

fig.add_annotation(
    x="2021",
    y=52,
    xshift=30,
    showarrow=False,
    text=f"49,2%",
    font=dict(size=12, color="#d62728", weight="bold")
)

fig.add_annotation(
    x="2021",
    y=5,
    xshift=30,
    showarrow=False,
    text=f"5,97%",
    font=dict(size=12, color="#ffcc00", weight="bold")
)

fig.add_annotation(
    x="2022",
    y=13,
    xshift=30,
    yshift=13,
    showarrow=False,
    text=f"13,8%",
    font=dict(size=12, color="#d62728", weight="bold")
)

fig.add_annotation(
    x="2022",
    y=39,
    xshift=30,
    yshift=12,
    showarrow=False,
    text=f"39,1%",
    font=dict(size=12, color="#ffcc00", weight="bold")
)
fig.add_annotation(
    x="2024",
    xshift=-50,
    y=37,
    showarrow=False,
    text=f"Indifferent",
    align="right",
    font=dict(size=12, color="#ffcc00", weight="bold")
)
fig.add_annotation(
    x="2024",
    xshift=-50,
    y=12.5,
    showarrow=False,
    text=f"Negativ",
    align="right",
    font=dict(size=12, color="#d62728", weight="bold")
)
fig.add_annotation(
    x="2024",
    xshift=-50,
    y=60.5,
    showarrow=False,
    text=f"Positiv",
    align="right",
    font=dict(size=12, color="grey", weight="bold")
)

fig.update_traces(line=dict(width=5))

fig.update_layout(
    xaxis=dict(title="", tickmode="array", tickvals=["2019", "2020", "2021", "2022", "2023", "2024"],range=["2019","2024"]),
    yaxis=dict(title="Selbsteinschätzung (%)", range=[0, 75]),
    showlegend=False,
    height=500,
    width=1500
)

fig.show()
fig.write_image("Plots/pdf/leistung2.pdf")